In [1]:
import os

from bs4 import BeautifulSoup
import requests

import pandas as pd
import numpy as np

In [2]:
baseURL = 'https://www.basketball-reference.com/leagues/NBA_'
seasons = [str(year) for year in range(1991, 2020)]
seasons = [year + '_per_game.html' for year in seasons]
for season in seasons:
    print(season)

1991_per_game.html
1992_per_game.html
1993_per_game.html
1994_per_game.html
1995_per_game.html
1996_per_game.html
1997_per_game.html
1998_per_game.html
1999_per_game.html
2000_per_game.html
2001_per_game.html
2002_per_game.html
2003_per_game.html
2004_per_game.html
2005_per_game.html
2006_per_game.html
2007_per_game.html
2008_per_game.html
2009_per_game.html
2010_per_game.html
2011_per_game.html
2012_per_game.html
2013_per_game.html
2014_per_game.html
2015_per_game.html
2016_per_game.html
2017_per_game.html
2018_per_game.html
2019_per_game.html


In [3]:
page = requests.get(baseURL + seasons[0])
soup = BeautifulSoup(page.content, 'html.parser')
categories = [th.getText() for th in soup.findAll('tr', limit=2)[0].findAll('th')]
categories = categories[1:]
for category in categories:
    print(category)

Player
Pos
Age
Tm
G
GS
MP
FG
FGA
FG%
3P
3PA
3P%
2P
2PA
2P%
eFG%
FT
FTA
FT%
ORB
DRB
TRB
AST
STL
BLK
TOV
PF
PTS


In [4]:
def getStats(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    rows = soup.findAll('tr')[1:]
    stats = [[row.getText() for row in rows[i].findAll('td')] for i in range(len(rows))]
    df = pd.DataFrame(stats, columns=categories)
    df['Player'] = df['Player'].str.replace('[^a-z A-Z|šđčćž|ŠĐČĆŽ]', '')
    df.dropna(how='all', axis=0, inplace=True)
    df.replace('', np.round(0.0000, 3), inplace=True)
    df = df[df['Tm'] != "TOT"]
    df['Tm'] = df['Tm'].str.replace('CHH', 'CHA')
    df['Tm'] = df['Tm'].str.replace('CHO', 'CHA')
    df['Tm'] = df['Tm'].str.replace('PHO', 'PHX')
    df['Tm'] = df['Tm'].str.replace('BRK', 'BKN')
    return df

In [5]:
def createCSVs():
    savePath = '/Users/pranav/nba_allNBA_predictor/season_statistics/'
    for season in seasons:
        df = getStats(baseURL + season)
        df.to_csv(savePath + season[0:4] + '_stats.csv', index=False)

In [6]:
createCSVs()

In [7]:
df = getStats(baseURL + seasons[0])

In [8]:
df.head(15)

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Alaa Abdelnaby,PF,22,POR,43,0,6.7,1.3,2.7,.474,...,.568,0.6,1.4,2.1,0.3,0.1,0.3,0.5,0.9,3.1
1,Mahmoud AbdulRauf,PG,21,DEN,67,19,22.5,6.2,15.1,.413,...,.857,0.5,1.3,1.8,3.1,0.8,0.1,1.6,2.2,14.1
2,Mark Acres,C,28,ORL,68,0,19.3,1.6,3.1,.509,...,.653,2.1,3.2,5.3,0.4,0.4,0.4,0.6,3.2,4.2
3,Michael Adams,PG,28,DEN,66,66,35.5,8.5,21.5,.394,...,.879,0.9,3.0,3.9,10.5,2.2,0.1,3.6,2.5,26.5
4,Mark Aguirre,SF,31,DET,78,13,25.7,5.4,11.7,.462,...,.757,1.7,3.1,4.8,1.8,0.6,0.3,1.6,2.7,14.2
5,Danny Ainge,SG,31,POR,80,0,21.4,4.2,8.9,.472,...,.826,0.6,2.0,2.6,3.6,0.8,0.2,1.3,2.4,11.1
6,Mark Alarie,PF,27,WSB,42,1,14.0,2.4,5.4,.440,...,.854,1.0,1.8,2.8,1.1,0.4,0.2,1.0,2.1,5.8
7,Steve Alford,PG,26,DAL,34,0,6.9,1.7,3.4,.504,...,.839,0.3,0.4,0.7,0.6,0.2,0.0,0.5,0.3,4.4
9,Greg Anderson,PF,26,MIL,26,0,9.5,1.0,2.8,.370,...,.571,1.0,1.9,2.9,0.1,0.3,0.3,0.8,1.1,2.7
10,Greg Anderson,PF,26,NJN,1,0,18.0,4.0,4.0,1.000,...,0,4.0,2.0,6.0,1.0,2.0,0.0,1.0,4.0,8.0
